In [48]:
import requests
import geopandas as gpd
from tqdm import tqdm

In [60]:
GEOELECTORAL = "https://geoelectoral.oep.org.bo/oep/rest/services"
ITEM = "AsientosElectorales/AsientosRecintos_20_05_2025"
OUTPUT = "datos"

In [67]:
def downloadLayer(mapserver, fieldMap, filename):
    print(filename)
    features = []
    featuresPerCall = 100
    server = f"{GEOELECTORAL}/{ITEM}/MapServer/{mapserver}"
    crs = requests.get(f"{server}?f=json").json()["sourceSpatialReference"]["wkid"]
    fields = ",".join(fieldMap.keys())
    featureIds = requests.get(
        f"{server}/query?where=1=1&returnIdsOnly=true&f=json"
    ).json()["objectIds"]
    for i in tqdm(range(0, len(featureIds), featuresPerCall)):
        features.extend(
            requests.get(
                f"{server}/query",
                params={
                    "objectIds": ",".join(
                        map(str, featureIds[i : i + featuresPerCall])
                    ),
                    "outFields": fields,
                    "returnGeometry": "true",
                    "f": "geojson",
                },
            ).json()["features"]
        )
    gpd.GeoDataFrame.from_features(features, crs=crs).rename(columns=fieldMap).to_file(
        f"{OUTPUT}/{filename}.gpkg", driver="GPKG"
    )

In [68]:
for mapserver, fieldMap, filename in zip(
    [0, 1],
    [{"IdLoc": "idloc"}, {"IdLoc": "idloc", "Reci": "recinto_codigo"}],
    ["asientos", "recintos"],
):
    downloadLayer(mapserver, fieldMap, filename)

asientos


100%|██████████| 38/38 [00:19<00:00,  1.95calls/s]


recintos


100%|██████████| 58/58 [00:21<00:00,  2.65calls/s]
